In [ ]:
import requests
from google.colab import userdata
from datetime import datetime
api_key = userdata.get("OPENAI_API_KEY")

In [ ]:
# List all models
r = requests.get("https://aiproxy.sanand.workers.dev/openai/v1/models", headers={
    "Authorization": f"Bearer {api_key}"
})
models=sorted(r.json()["data"], key=lambda x: x['created'], reverse=True)

In [ ]:
points = 0
tts_1_1106_created_date = datetime(2023, 4, 19).date()
tts_1_1106_found = False
gpt_4o_mini_at_index_1 = False
text_embedding_index = None
gpt_35_turbo_index = None
# Iterate once through the list to gather relevant details
for i, model in enumerate(models):
    if model["id"] == "tts-1-1106":
        # Check Condition 1
        tts_creation_date = datetime.fromtimestamp(model["created"]).date()
        if tts_creation_date == tts_1_1106_created_date:
            points += 4
        tts_1_1106_found = True
    elif i == 1 and model["id"] == "gpt-4o-mini":
        # Check Condition 2
        points += 2
        gpt_4o_mini_at_index_1 = True
    elif model["id"] == "text-embedding-3-large":
        # Save index for Condition 3
        text_embedding_index = i
    elif model["id"] == "gpt-3.5-turbo":
        # Save index for Condition 3
        gpt_35_turbo_index = i
# Final check for Condition 3 if both indices are found
if text_embedding_index is not None and gpt_35_turbo_index is not None:
    if text_embedding_index == gpt_35_turbo_index - 16:
        points += 1
# Output the total points
print("Total points:", points)

Total points: 0


In [ ]:
cutoff_date = datetime(2024, 11, 30).timestamp()
filtered_models = sorted(
    [model for model in models if model["created"] < cutoff_date],
    key=lambda x: x["created"],
    reverse=True)
# Initialize points
points = 0
# Condition 1: Check if whisper-1 was created on 2023-04-19
for model in filtered_models:
    if model["id"] == "whisper-1":
        creation_date = datetime.fromtimestamp(model["created"]).date()
        if creation_date == datetime(2023, 4, 19).date():
            points += 4
        break
# Condition 2: Check if gpt-3.5-turbo is located at index 8
if len(filtered_models) > 8 and filtered_models[8]["id"] == "gpt-3.5-turbo":
    points += 2
# Condition 3: Check if gpt-3.5-turbo-instruct was created 10 models before tts-1
gpt_35_instruct_index = next((i for i, m in enumerate(filtered_models) if m["id"] == "gpt-3.5-turbo-instruct"), None)
tts_1_index = next((i for i, m in enumerate(filtered_models) if m["id"] == "tts-1"), None)
if gpt_35_instruct_index is not None and tts_1_index is not None:
    if gpt_35_instruct_index == tts_1_index - 10:
        points += 1
# Output the total points
print("Total points:", points)

Total points: 0


In [ ]:
import requests
import json

# Define the prompt
prompt = """
I have a list of 20 conference attendees. Please generate their data in the following JSON structure:

{
  "fn": "Full name",
  "bday": "Birthday (YYYY-MM-DD format)",
  "email": "Email address",
  "tel": "Telephone number (nnn-nnn-nnnn format)",
  "adr": {
    "country-name": "Country"
  },
  "org": "Organization name",
  "title": "Job title",
  "photo": "Photograph (URL or placeholder)",
  "url": "URL",
  "nickname": "Nickname"
}

Here are the attendees:
1. James Allen - Email: james.a@example.com, Phone: 234-567-8901. Nickname: Jim. Birthday: October 25, 1982.
2. Ethan Turner - Email: ethan.t@example.com, Phone: 567-890-1234, Canada.
3. Daniel Young - Engineer at Tech Dynamics, USA. Phone: 555-987-6543. Nickname: Dan. Birthday: March 3, 1990.
4. David Miller - Consultant. Nickname: Dave.
5. Aiden Edwards - UK. Email: aiden.e@example.com. Nickname: Aid.
6. Jack Wright - Designer, UK. Nickname: Jackie. Birthday: April 12, 1985.
7. Emily Hughes - Designer, Canada. Nickname: Em. Birthday: January 20, 1984.
8. John Doe - Manager at Acme Corp, USA. Email: john.doe@example.com, Phone: 123-456-7890, Website: https://example.com/johnd, Nickname: Johnny. Birthday: January 15, 1990.
9. Megan Thomas - Canada. Nickname: Meg.
10. Matthew White - Researcher, Australia. Nickname: Matt.
11. Emily Davis - Analyst, UK. Nickname: Em. Birthday: March 25, 1992.
12. Isabella King - Australia. Email: isabella.k@example.com. Nickname: Bella.
13. Sophia Young - Canada. Nickname: Soph. Birthday: September 10, 1998.
14. William Carter - Nickname: Will.
15. Grace Baker - Creative Solutions, Australia.
16. Amelia Scott - Nickname: Amy.
17. Daniel Harris - Engineer. Nickname: Dan.
18. Emily Hill - Developer, USA. Nickname: Em. Birthday: June 17, 1996.
19. Laura Taylor.
20. Lucas Perez - Nickname: Luke.

Please return the data in a JSON array with each attendee in the defined format.
"""

# Set up the headers and the data to send in the POST request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

data = {
    "model": "gpt-4o",  # Or use "gpt-3.5-turbo" if you have access
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.5,
    "max_tokens": 2000  # You can adjust the number of tokens if needed
}

# Make the POST request to OpenAI API
response = requests.post(
    "https://aiproxy.sanand.workers.dev/openai/v1/models",
    headers=headers,
    data=json.dumps(data)
)

# Check for successful response and print the result
if response.status_code == 200:
    # Get the generated content
    generated_json = response.json()
    attendee_data = generated_json['choices'][0]['message']['content']
    print(attendee_data)
else:
    print(f"Error: {response.status_code} - {response.text}")


Error: 400 - {
  "message": "Invalid model: gpt-4o"
}


In [ ]:
import requests
import numpy as np

api_key = userdata.get("OPENAI_API_KEY")

# Define the model and input
model = "text-embedding-3-small"  # or "text-embedding-3-small" if you prefer that model
input_text = "Economic"

# Send a POST request to get the embedding
response = requests.post(
    "https://aiproxy.sanand.workers.dev/openai/v1/embeddings",
    headers={"Authorization": f"Bearer {api_key}"},
    json={
        "model": model,
        "input": input_text
    }
)

# Check if the response is successful
if response.status_code == 200:
    # Extract the embedding vector from the response
    embedding = np.array(response.json()['data'][0]['embedding'])

    # Define the threshold to compare against
    threshold = -0.027582391942094515

    # Count how many values in the embedding are greater than the threshold
    count = np.sum(embedding > threshold)

    print(f"Number of values greater than {threshold}: {count}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Number of values greater than -0.027582391942094515: 1350


In [ ]:
import requests
import numpy as np
from numpy.linalg import norm

# Define the model and input words
model = "text-embedding-3-small"  # Correct model for embeddings
words = ["Champion", "Flexible"]

# Function to get the embedding of a word
def get_embedding(word):
    response = requests.post(
        "https://aiproxy.sanand.workers.dev/openai/v1/embeddings",
        headers={"Authorization": f"Bearer {api_key}"},
        json={"model": model, "input": word}
    )

    if response.status_code == 200:
        # Extract and return the embedding from the response
        return np.array(response.json()['data'][0]['embedding'])
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Get the embeddings for both words
embedding_champion = get_embedding("Champion")
embedding_flexible = get_embedding("Flexible")

# Check if both embeddings were retrieved successfully
if embedding_champion is not None and embedding_flexible is not None:
    # Calculate cosine similarity
    cosine_similarity = np.dot(embedding_champion, embedding_flexible) / (norm(embedding_champion) * norm(embedding_flexible))

    print(f"Cosine Similarity between 'Champion' and 'Flexible': {cosine_similarity}")
else:
    print("Failed to retrieve embeddings for both words.")


Cosine Similarity between 'Champion' and 'Flexible': 0.20113555266733715


In [ ]:
import requests
import numpy as np
from numpy.linalg import norm

# Define the model
model = "text-embedding-3-small"

# Word lists to evaluate
word_lists = {
    'a': ["Important", "Champion", "Different", "Priority"],
    'b': ["Flexible", "Priority", "Ambitious", "Evolution"],
    'c': ["Champion", "Discovery", "Industry", "Universe"],
    'd': ["Establish", "Original", "Positive", "Agreement"]
}

# Function to get the embedding of a word
def get_embedding(word):
    response = requests.post(
        "https://aiproxy.sanand.workers.dev/openai/v1/embeddings",
        headers={"Authorization": f"Bearer {api_key}"},
        json={"model": model, "input": word}
    )
    if response.status_code == 200:
        return np.array(response.json()['data'][0]['embedding'])
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Function to calculate cosine similarity
def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))

# Get the embedding for the word "Discovery"
discovery_embedding = get_embedding("Discovery")

# Dictionary to store the average cosine similarity for each word list
average_similarities = {}

# Loop over each word list and calculate the average cosine similarity with "Discovery"
for list_name, word_list in word_lists.items():
    similarities = []
    for word in word_list:
        word_embedding = get_embedding(word)
        if word_embedding is not None and discovery_embedding is not None:
            similarity = cosine_similarity(discovery_embedding, word_embedding)
            similarities.append(similarity)
    # Calculate and store the average cosine similarity for this list
    if similarities:
        average_similarities[list_name] = np.mean(similarities)

# Find the list with the lowest average cosine similarity
lowest_similarity_list = min(average_similarities, key=average_similarities.get)

print(f"The list with the lowest average cosine similarity with 'Discovery' is: {lowest_similarity_list}")


The list with the lowest average cosine similarity with 'Discovery' is: a


In [ ]:
import requests
import json

# Text to analyze
text = "Un perro ladra ruidosamente mientras persigue a una ardilla por el patio trasero."


# Endpoint for tokenizing text
url = "https://aiproxy.sanand.workers.dev/openai/v1/embeddings"

# Request headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Data to send to OpenAI's tokenizer
data = {
    "model": "gpt-3.5-turbo-0125",  # Use any embedding model
    "input": text
}

# Send the request
response = requests.post(url, headers=headers, json=data)

# Parse the response
response_json = response.json()

# Count the tokens
if "usage" in response_json:
    tokens = response_json["usage"]["total_tokens"]
else:
    print("Error: Could not tokenize the text")
    tokens = 0

# Cost per million tokens (50 cents)
cost_per_million_tokens = 0.50

# Calculate the cost in cents
cost_in_cents = (tokens / 1_000_000) * cost_per_million_tokens

# Print the result
print(f"Number of tokens: {tokens}")
print(f"Cost in cents: {cost_in_cents}")


Error: Could not tokenize the text
Number of tokens: 0
Cost in cents: 0.0


In [ ]:
import requests
import json

# Define the headers
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Define the prompt
prompt = """
Here is a list of 20 attendees to a conference. Convert them into a JSON array of objects with these fields (consistent with vCard):

fn: Full name.
bday: Birthday (YYYY-MM-DD format)
email: Email address.
tel: Telephone number (nnn-nnn-nnnn format)
adr.country-name: Country. (Note that adr is an object and country-name is a key inside that)
org: Organization name.
title: Job title.
photo: Photograph.
url: URL.
nickname: Nickname.

Here are the attendees:

1 James Clark - Analyst, USA. Phone: 321-888-4444. Nickname: Jim.
2 Daniel Young - Engineer at Tech Dynamics, USA. Phone: 555-987-6543. Nickname: Dan. Birthday: March 3, 1990.
3 Anthony Lewis - Email: anthony.l@example.com, Phone: 789-012-3456. Nickname: Tony.
4 Aiden Edwards - UK. Email: aiden.e@example.com. Nickname: Aid.
5 Amelia Scott - Nickname: Amy.
6 Michael Brown - Manager at Tech Solutions, USA. Nickname: Mike.
7 Emma Green - Email: emma.g@example.com, Phone: 456-789-0123. Nickname: Em.
8 Natalie King - Future Tech, USA. Nickname: Nat.
9 Alex Johnson - Developer at XYZ Inc., Canada. Email: alex.j@example.com, Phone: 987-654-3210, Website: https://xyz.com/alexj, Nickname: AJ. Birthday: February 20, 1985.
10 Michael Harris - Project Manager at Future Tech, Canada. Nickname: Mike.
11 Ava Mitchell - Developer, USA. Email: ava.m@example.com, Phone: 123-555-7890, Website: https://example.com/avam. Birthday: December 12, 1994.
12 Isabella King - Australia. Email: isabella.k@example.com. Nickname: Bella.
13 Emily Hill - Developer, USA. Nickname: Em. Birthday: June 17, 1996.
14 Mia Robinson - Manager at ABC Corp, UK. Phone: 987-111-3210. Nickname: Mimi. Birthday: January 22, 1987.
15 Grace Baker - Creative Solutions, Australia.
16 Jessica Martin - Email: jessica.m@example.com, Phone: 654-987-3210, Website: https://example.com/jessicam, Nickname: Jess. Birthday: July 20, 1993.
17 Joshua Ward - Australia. Nickname: Josh.
18 Lucas Perez - Nickname: Luke.
19 Emily Hughes - Designer, Canada. Nickname: Em. Birthday: January 20, 1984.
20 Matthew White - Researcher, Australia. Nickname: Matt.
"""

# Define the request body
data = {
    "model": "gpt-4o-mini",  # or "gpt-4"
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
}

# Send the POST request to OpenAI's API
response = requests.post(
    "https://aiproxy.sanand.workers.dev/openai/v1/chat/completions",  # Correct endpoint for chat completions
    headers=headers,
    data=json.dumps(data)
)

# Parse the response
generated_response = response.json()

# Extract and clean up the response to match desired format
attendees = generated_response['choices'][0]['message']['content']

# Replace `null` with empty string in the JSON response
attendees_json = json.loads(attendees)  # Converts the string response into a Python list

# Modify the data to replace `null` values with empty strings where appropriate
for attendee in attendees_json:
    for key in attendee:
        if attendee[key] is None:
            attendee[key] = ""

# Print the cleaned-up JSON
print(json.dumps(attendees_json, indent=4))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
import json

# Define the endpoint and headers
url = 'https://api.openai.com/v1/engines/text-davinci-003/completions'
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

# Define the data (prompt) for the request
data = {
    "prompt": "Hello, how are you?",
    "max_tokens": 50,
    "temperature": 0.5
}

# Send a POST request to the OpenAI API
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.status_code == 200:
    # Parse the response to get the number of tokens used
    response_data = response.json()
    tokens_used = response_data['usage']['total_tokens']
    print(f"Tokens used: {tokens_used}")
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


Request failed with status code 401: {
    "error": {
        "message": "Your authentication token is not from a valid issuer.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_issuer"
    }
}



In [1]:
AIPROXY_TOKEN=1
print(AIPROXY_TOKEN)

1
